# INTELLIGENT RAIL

## Edward Jackson

###### Notebook 1 of 6 (feature engineering - maintenance data)

In [432]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from google_trans_new import google_translator  

In [433]:
data = pd.read_csv('data/interceptions_programmees_sur_ligne.csv', delimiter=';')

### Data wrangling - network maintenance data

In [434]:
data.shape

(98037, 9)

In [435]:
data.sample(10)

,LIB_STRUCTDEM,COD_LIGNE,LIB_LIGNE,PK_DEBM,PK_FINM,FAMILLETRAVAUX,NB_INTERVENTIONS,NUM_SEMAINE,ANNEE
94618,Siège INFRAPOLE Paris Rive Gauche,570000,Ligne de Paris-Austerlitz à Bordeaux-St-Jean,0,82531,entretien des caténaires,20,27.0,2017.0
13187,Siège INFRAPOLE Est-Européen,5000,Ligne de Paris à Strasbourg (LGV),0,406015,entretien et travaux divers,2,26.0,2020.0
85295,Siège INFRAPOLE Alpes,880000,Ligne de Mouchard à Bourg-en-Bresse,478625,506164,entretien des caténaires,2,17.0,2017.0
19932,Siège INFRAPOLE PACA,946000,Ligne de Coni à Vintimille,31800,99388,entretien de la signalisation,1,36.0,2020.0
8384,Siège INFRAPOLE Pays-de-Loire,470000,Ligne de Savenay à Landerneau,469737,504508,entretien de la signalisation,4,2.0,2020.0
32242,Siège INFRAPOLE Languedoc-Roussillon,810000,Ligne de Tarascon à Sète-Ville,0,800,entretien de la signalisation,9,40.0,2019.0
92248,Siège INFRAPOLE Champagne-Ardenne,12000,Ligne de Troyes à Brienne-le-Château,169997,207800,entretien de la voie,2,28.0,2017.0
66759,Siège INFRAPOLE Champagne-Ardenne,204000,Ligne de Mohon à Thionville,-58,193021,entretien et travaux divers,22,42.0,2018.0
82840,Siège INFRAPOLE Rhodanien,890000,Ligne de Lyon-Perrache à Genève (frontière),8120,43428,entretien et travaux divers,3,47.0,2017.0
64728,Siège INFRAPOLE Aquitaine,655000,Ligne de Bordeaux-St-Jean à Irun,0,95310,entretien des caténaires,37,23.0,2018.0


#### Data dictionary

There are 98,037 rows across 9 columns:
- **LIB_STRUCTDEM** - the regional headquarters responsible for maintenance in an area
- **COD_LIGNE** - the unique track numbers (generally speaking, the last 3 digits refer to specific features/sections on a line and leading digits reference a complete line)
- **LIB_LIGNE** - a locator description of the section of track
- **PK_DEBM** and **PK_FINM** - start/finish reference markers on the line
- **FAMILLETRAVAUX** - category of works
- **NB_INTERVENTIONS** - duration of works
- **NUM_SEMAINE** - week number when works commenced
- **ANNEE** - year

In [436]:
data['LIB_STRUCTDEM'].unique()

array(['Siège INFRAPOLE Alpes', 'Siège INFRAPOLE Aquitaine',
       'Siège INFRAPOLE Auvergne Nivernais',
       'Siège INFRAPOLE Bourgogne Franche-Comté',
       'Siège INFRAPOLE Bretagne', 'Siège INFRAPOLE Centre',
       'Siège INFRAPOLE Champagne-Ardenne',
       'Siège INFRAPOLE Haute-Picardie', 'Siège INFRAPOLE Est-Européen',
       'Siège INFRAPOLE Indre Limousin',
       'Siège INFRAPOLE Languedoc-Roussillon',
       'Siège INFRAPOLE LGV Sud Est Européen', 'Siège INFRAPOLE Lorraine',
       'Siège INFRAPOLE Midi-Pyrénées',
       'Siège INFRAPOLE Nord-Pas-de-Calais', 'Siège INFRAPOLE Normandie',
       'Siège INFRAPOLE PACA', 'Siège INFRAPOLE Paris Saint-Lazare',
       'Siège INFRAPOLE Paris-Est', 'Siège INFRAPOLE Paris-Nord',
       'Siège INFRAPOLE Pays-de-Loire', 'Siège INFRAPOLE Paris-Sud-Est',
       'Siège INFRAPOLE Poitou-Charentes', 'Siège INFRAPOLE Rhénan',
       'Siège INFRAPOLE Rhodanien', 'Siège INFRAPOLE LGV Atlantique',
       'Siège INFRAPOLE Nord Européen',
  

We can see that a number of the French regions are embedded in the first column. Alongside the data in **COD_LIGNE**, this could be developed to provide a key to join with the punctuality data.

In [437]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98037 entries, 0 to 98036
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LIB_STRUCTDEM     98037 non-null  object 
 1   COD_LIGNE         98037 non-null  int64  
 2   LIB_LIGNE         98037 non-null  object 
 3   PK_DEBM           98037 non-null  int64  
 4   PK_FINM           98037 non-null  int64  
 5   FAMILLETRAVAUX    98037 non-null  object 
 6   NB_INTERVENTIONS  98037 non-null  int64  
 7   NUM_SEMAINE       96904 non-null  float64
 8   ANNEE             96904 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 6.7+ MB


In order to proceed with this initial cleaning, we need to set the data type of our line codes to *string* as they are labels rather than continuous values.

In [438]:
data['COD_LIGNE'] = data['COD_LIGNE'].astype('str')

We already know the lines used on the high-speed network. Notice that most are suffixed with '000'. These will need to be updated to be compatible with the line codes in our other data.

In [439]:
# line codes known from cleaning of line data
lgv_lines = ['905000', '830000', '1000', '70000', '750000', '640000', '752000',
       '897000', '14000', '900000', '677000', '420000', '930000',
       '890000', '515000', '538000', '850000', '468000', '226000',
       '441000', '470000', '278000', '429000', '431000', '89000', '5000',
       '301000', '752100', '262000', '450000', '216000', '226310',
       '834000', '768300']

In [440]:
lgv_rows = data[data['COD_LIGNE'].isin(lgv_lines)]

In [441]:
lgv_rows.head(5)

,LIB_STRUCTDEM,COD_LIGNE,LIB_LIGNE,PK_DEBM,PK_FINM,FAMILLETRAVAUX,NB_INTERVENTIONS,NUM_SEMAINE,ANNEE
0,Siège INFRAPOLE Alpes,900000,Ligne de Culoz à Modane (frontière),100845,247316,entretien de la voie,25,35.0,2020.0
12,Siège INFRAPOLE Bourgogne Franche-Comté,830000,Ligne de Paris-Lyon à Marseille-St-Charles,219609,455512,création ou modernisation de ligne,2,35.0,2020.0
14,Siège INFRAPOLE Bretagne,441000,Ligne de Rennes à St-Malo-St-Servan,375681,454364,entretien de la voie,4,35.0,2020.0
18,Siège INFRAPOLE Bretagne,470000,Ligne de Savenay à Landerneau,469470,768834,entretien des caténaires,6,35.0,2020.0
22,Siège INFRAPOLE Champagne-Ardenne,70000,Ligne de Noisy-le-Sec à Strasbourg-Ville,111739,231893,renouvellement de la signalisation,8,35.0,2020.0


In [442]:
lgv_rows.shape

(24592, 9)

We see that 24,592 rows of our maintenance data are associated with lines on the high-speed rail network.

We can proceed with this filtered data. First, we extract the region labels in the strings within the **LIB_STRUCTDEM** column:

In [443]:
lgv_rows['LIB_STRUCTDEM'] = lgv_rows['LIB_STRUCTDEM'].str.replace('Siège INFRAPOLE ','')

<ipython-input-443-d3daa99a784c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lgv_rows['LIB_STRUCTDEM'] = lgv_rows['LIB_STRUCTDEM'].str.replace('Siège INFRAPOLE ','')


In [444]:
lgv_rows.head()

,LIB_STRUCTDEM,COD_LIGNE,LIB_LIGNE,PK_DEBM,PK_FINM,FAMILLETRAVAUX,NB_INTERVENTIONS,NUM_SEMAINE,ANNEE
0,Alpes,900000,Ligne de Culoz à Modane (frontière),100845,247316,entretien de la voie,25,35.0,2020.0
12,Bourgogne Franche-Comté,830000,Ligne de Paris-Lyon à Marseille-St-Charles,219609,455512,création ou modernisation de ligne,2,35.0,2020.0
14,Bretagne,441000,Ligne de Rennes à St-Malo-St-Servan,375681,454364,entretien de la voie,4,35.0,2020.0
18,Bretagne,470000,Ligne de Savenay à Landerneau,469470,768834,entretien des caténaires,6,35.0,2020.0
22,Champagne-Ardenne,70000,Ligne de Noisy-le-Sec à Strasbourg-Ville,111739,231893,renouvellement de la signalisation,8,35.0,2020.0


In [445]:
lgv_rows['LIB_STRUCTDEM'].unique()

array(['Alpes', 'Bourgogne Franche-Comté', 'Bretagne',
       'Champagne-Ardenne', 'Est-Européen', 'Languedoc-Roussillon',
       'LGV Sud Est Européen', 'Lorraine', 'PACA', 'Paris-Est',
       'Paris-Sud-Est', 'Pays-de-Loire', 'Rhénan', 'Rhodanien',
       'LGV Atlantique', 'Midi-Pyrénées', 'Poitou-Charentes', 'Aquitaine',
       'Auvergne Nivernais', 'Centre', 'Nord-Pas-de-Calais',
       'Nord Européen', 'Paris-Nord', 'Ouest Parisien'], dtype=object)

We see that not all French regions are respresented here. However, a quick internet search revealed that some of the headquarters in a region are branded. Therefore, using this information, we can revert to the actual region names. Note that some headquarters cover more than one region. This is unavoidable but it is mitigated somewhat by network sections having two reference points in the final data: line code AND region.

In [446]:
lgv_rows['LIB_STRUCTDEM'].replace({'Est-Européen':'Franche-Comté',
                                   'Est-Européen':'Bourgogne',
                                   'Bourgogne Franche-Comté':'Bourgogne',
                                   'Bourgogne Franche-Comté':'Franche-Comté',
                                   'Paris-Nord':'Île-de-France',
                                   'Ouest Parisien':'Île-de-France',
                                   'Paris-Est':'Île-de-France',
                                   'Paris-Sud-Est':'Île-de-France',
                                   'Nord Européen':'Picardie',
                                   'Nord Européen':'Île-de-France',
                                   'Auvergne Nivernais':'Auvergne',
                                   'LGV Sud Est Européen':'Centre',
                                   'LGV Sud Est Européen':'Auvergne',
                                   'LGV Sud Est Européen':'Languedoc-Roussillon',
                                   'LGV Sud Est Européen':'Rhône-Alpes',
                                   'LGV Atlantique':'Centre',
                                   'LGV Atlantique':'Poitou-Charentes',
                                   'LGV Atlantique':'Aquitaine',
                                   'LGV Atlantique':'Midi-Pyrénées',
                                   'PACA':"Provence-Alpes-Côte-d'Azur",
                                   'Rhénan':"Alsace",
                                   'Rhodanien':'Auvergne',
                                   'Rhodanien':'Languedoc-Roussillon',
                                   'Rhodanien':'Rhône-Alpes',
                                   'Pays-de-Loire':"Pays de la Loire",
                                   'Alpes':'Rhône-Alpes'}, inplace=True)

C:\Users\epjac\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [447]:
lgv_rows['LIB_STRUCTDEM'].unique()

array(['Rhône-Alpes', 'Franche-Comté', 'Bretagne', 'Champagne-Ardenne',
       'Bourgogne', 'Languedoc-Roussillon', 'Lorraine',
       "Provence-Alpes-Côte-d'Azur", 'Île-de-France', 'Pays de la Loire',
       'Alsace', 'Midi-Pyrénées', 'Poitou-Charentes', 'Aquitaine',
       'Auvergne', 'Centre', 'Nord-Pas-de-Calais'], dtype=object)

In [448]:
# update column name
lgv_rows = lgv_rows.rename(columns=({'LIB_STRUCTDEM':'Region'}))

In [449]:
lgv_rows.head()

,Region,COD_LIGNE,LIB_LIGNE,PK_DEBM,PK_FINM,FAMILLETRAVAUX,NB_INTERVENTIONS,NUM_SEMAINE,ANNEE
0,Rhône-Alpes,900000,Ligne de Culoz à Modane (frontière),100845,247316,entretien de la voie,25,35.0,2020.0
12,Franche-Comté,830000,Ligne de Paris-Lyon à Marseille-St-Charles,219609,455512,création ou modernisation de ligne,2,35.0,2020.0
14,Bretagne,441000,Ligne de Rennes à St-Malo-St-Servan,375681,454364,entretien de la voie,4,35.0,2020.0
18,Bretagne,470000,Ligne de Savenay à Landerneau,469470,768834,entretien des caténaires,6,35.0,2020.0
22,Champagne-Ardenne,70000,Ligne de Noisy-le-Sec à Strasbourg-Ville,111739,231893,renouvellement de la signalisation,8,35.0,2020.0


In [450]:
lgv_rows.shape

(24592, 9)

We see that we now have all French regions correctly labelled in our maintenance data.

A quick internet search reveals that columns **PK_DEBM** and **PK_FINM** refer to *points kilometrique* or waymarkers for line features. We will not require this granularity of detail for the project so we will remove the two columns now.

In [451]:
lgv_rows = lgv_rows.drop(columns=['PK_DEBM', 'PK_FINM'])

Next, we tackle the French language descriptions of works in **FAMILLETRAVAUX**. Using the Google Translate API, we can translate these for our purposes:

In [452]:
lgv_rows['FAMILLETRAVAUX'].unique()

array(['entretien de la voie', 'création ou modernisation de ligne',
       'entretien des caténaires', 'renouvellement de la signalisation',
       'entretien et travaux divers', 'entretien de la signalisation',
       'renouvellement de ponts, tunnels et talus',
       'travaux pour des tiers', 'renouvellement de la voie',
       'entretien de ponts, tunnels et talus',
       'renouvellement des caténaires',
       'renouvellement de la télécommunication ferroviaire'], dtype=object)

In [453]:
translator = google_translator()

In [454]:
# store unique maintenance headings in list (French language)
french = lgv_rows['FAMILLETRAVAUX'].unique()

# create empty dictionary object
vocab = dict()

# looping through each French phrase
for i in range(len(french)):
    
    # each French phrase translated to English and stored
    english = translator.translate(french[i])
    
    # French-to-English pair added to dictionary object
    vocab.update({french[i]:english}) 

We create a *French-to-English* dictionary and use it to update our entries.

In [455]:
vocab

{'entretien de la voie': 'Maintenance of the way ',
 'création ou modernisation de ligne': 'Creation or modernization of the line ',
 'entretien des caténaires': 'Catenary maintenance ',
 'renouvellement de la signalisation': 'Renewal of signaling ',
 'entretien et travaux divers': 'Maintenance and various work ',
 'entretien de la signalisation': 'signaling maintenance ',
 'renouvellement de ponts, tunnels et talus': 'Renewal of bridges, tunnels and slopes ',
 'travaux pour des tiers': 'work for third parties ',
 'renouvellement de la voie': 'renewal of the way ',
 'entretien de ponts, tunnels et talus': 'Maintenance of bridges, tunnels and slopes ',
 'renouvellement des caténaires': 'Catenary renewal ',
 'renouvellement de la télécommunication ferroviaire': 'Renewal of railway telecommunication '}

In [456]:
# dictionary passed over column to 'translate' phrases as matched
lgv_rows["FAMILLETRAVAUX"] = lgv_rows["FAMILLETRAVAUX"].map(vocab)

In [457]:
lgv_rows.head(5)

,Region,COD_LIGNE,LIB_LIGNE,FAMILLETRAVAUX,NB_INTERVENTIONS,NUM_SEMAINE,ANNEE
0,Rhône-Alpes,900000,Ligne de Culoz à Modane (frontière),Maintenance of the way,25,35.0,2020.0
12,Franche-Comté,830000,Ligne de Paris-Lyon à Marseille-St-Charles,Creation or modernization of the line,2,35.0,2020.0
14,Bretagne,441000,Ligne de Rennes à St-Malo-St-Servan,Maintenance of the way,4,35.0,2020.0
18,Bretagne,470000,Ligne de Savenay à Landerneau,Catenary maintenance,6,35.0,2020.0
22,Champagne-Ardenne,70000,Ligne de Noisy-le-Sec à Strasbourg-Ville,Renewal of signaling,8,35.0,2020.0


The line description in column **LIB_LIGNE** is synonymous with the unique identifier code in **COD_LIGNE**. The line code will be a key for joining data later so we will drop the **LIB_LIGNE** column. (In developmeny, translating a whole dataframe via a loop also proved to be a computationally expensive exercise!)

In [458]:
lgv_rows = lgv_rows.drop(columns='LIB_LIGNE')

#### Missing values

In [459]:
lgv_rows.isna().sum().sum()

622

Given that there are 622 missing values, we can explore in more detail:

In [460]:
lgv_rows.isna().sum()

Region                0
COD_LIGNE             0
FAMILLETRAVAUX        0
NB_INTERVENTIONS      0
NUM_SEMAINE         311
ANNEE               311
dtype: int64

We check whether the same 311 rows are affected.

In [461]:
len(lgv_rows[lgv_rows.isna().any(axis=1)])

311

In [462]:
# percentage of rows affected
round(len(lgv_rows[lgv_rows.isna().any(axis=1)])/len(lgv_rows)*100, 2)

1.26

We see that 311 maintenance entries do not have a month or year attributed to them. Given that we have no way of retrieving this information easily and just ~1% of rows are affected, we drop these rows.

In [463]:
lgv_rows = lgv_rows.dropna()

In [464]:
lgv_rows.shape

(24281, 6)

Confirmation that we now have 24,281 rows remaining.

We notice that both the **NUM_SEMAINE** and **ANNEE** column values appear as floats. We now change their *dtypes* to INTEGER.

In [465]:
lgv_rows = lgv_rows.astype({'NUM_SEMAINE':'int', 'ANNEE':'int'})

We rename our columns for ease of reference moving forward.

In [466]:
lgv_rows.columns = ['region','line_code','works_category','works_duration','week_no','year']

In [467]:
lgv_rows.head()

,region,line_code,works_category,works_duration,week_no,year
0,Rhône-Alpes,900000,Maintenance of the way,25,35,2020
12,Franche-Comté,830000,Creation or modernization of the line,2,35,2020
14,Bretagne,441000,Maintenance of the way,4,35,2020
18,Bretagne,470000,Catenary maintenance,6,35,2020
22,Champagne-Ardenne,70000,Renewal of signaling,8,35,2020


Earlier, we saw that there were 12 different classes in **works_category**.

In [468]:
lgv_rows['works_category'].unique()

array(['Maintenance of the way ',
       'Creation or modernization of the line ', 'Catenary maintenance ',
       'Renewal of signaling ', 'Maintenance and various work ',
       'signaling maintenance ',
       'Renewal of bridges, tunnels and slopes ',
       'work for third parties ', 'renewal of the way ',
       'Maintenance of bridges, tunnels and slopes ', 'Catenary renewal ',
       'Renewal of railway telecommunication '], dtype=object)

Many are paired. We will one-hot encode each *unique* category to make the information compatible with machine learning models:
- Maintenance of the way, Renewal of the way ----> **track_works**
- Signaling maintenance, renewal of signaling ----> **signal_works**
- Catenary maintenance, catenary renewal ----> **overhead_works**
- Maintenance of bridges, tunnels, slopes, Renewal of bridges, tunnels, slopes ----> **land_maintenance**
- Maintenance and various work, Work for third parties ----> **other_works**
- Creation or modernisation of lines ----> **modernisation_works**
- Renewal of railway telecommunication ----> **comms_works**

In [469]:
# dictionary to categorise works
works_categories = {'Maintenance of the way ':'track_works',
                   'renewal of the way ':'track_works',
                   'Catenary maintenance ':'overhead_works',
                   'Maintenance of bridges, tunnels and slopes ':'land_works',
                   'Maintenance and various work ':'other_works',
                   'Creation or modernization of the line ':'modernisation_works',
                   'Renewal of signaling ':'signal_works',
                   'signaling maintenance ':'signal_works',
                   'Renewal of bridges, tunnels and slopes ':'land_works',
                   'work for third parties ':'other_works',
                   'Catenary renewal ':'overhead_works',
                   'Renewal of railway telecommunication ':'comms_works'}

We can update the column by mapping it with the dictionary above.

In [470]:
lgv_rows['works_category'] = lgv_rows['works_category'].map(works_categories)

In [471]:
lgv_rows['works_category'].unique()

array(['track_works', 'modernisation_works', 'overhead_works',
       'signal_works', 'other_works', 'land_works', 'comms_works'],
      dtype=object)

In [472]:
lgv_rows.head()

,region,line_code,works_category,works_duration,week_no,year
0,Rhône-Alpes,900000,track_works,25,35,2020
12,Franche-Comté,830000,modernisation_works,2,35,2020
14,Bretagne,441000,track_works,4,35,2020
18,Bretagne,470000,overhead_works,6,35,2020
22,Champagne-Ardenne,70000,signal_works,8,35,2020


In [473]:
lgv_rows.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24281 entries, 0 to 96894
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   region          24281 non-null  object
 1   line_code       24281 non-null  object
 2   works_category  24281 non-null  object
 3   works_duration  24281 non-null  int64 
 4   week_no         24281 non-null  int32 
 5   year            24281 non-null  int32 
dtypes: int32(2), int64(1), object(3)
memory usage: 1.1+ MB


For visualisations purposes, **year** must be an object rather than an integer (we wouldn't wish to view 2019-and-a-half!).

In [474]:
lgv_rows['year'] = lgv_rows['year'].astype('int')

We can now one-hot encode our works categories and drop the **works_category** source.

In [475]:
lgv_rows = pd.concat([lgv_rows, pd.get_dummies(lgv_rows['works_category'])], axis=1)

In [476]:
lgv_rows = lgv_rows.drop(columns='works_category')

In [477]:
lgv_rows.head()

,region,line_code,works_duration,week_no,year,comms_works,land_works,modernisation_works,other_works,overhead_works,signal_works,track_works
0,Rhône-Alpes,900000,25,35,2020,0,0,0,0,0,0,1
12,Franche-Comté,830000,2,35,2020,0,0,1,0,0,0,0
14,Bretagne,441000,4,35,2020,0,0,0,0,0,0,1
18,Bretagne,470000,6,35,2020,0,0,0,0,1,0,0
22,Champagne-Ardenne,70000,8,35,2020,0,0,0,0,0,1,0


Works are still grouped by weeks in the year. Given that our punctuality data breaks routes down into months, we will transform our weekly maintenance records to monthly. Although not a perfect solution, we will *bin* each quarter of a year (i.e. 13 weeks) into three months of 4, 4 and 5 weeks.

In [478]:
print('Smallest week number: ',lgv_rows['week_no'].min())
print('Largest week number: ',lgv_rows['week_no'].max())

Smallest week number:  1
Largest week number:  52


In [479]:
bins = [0,4,8,13, 17,21,26, 30,34,39, 43,47,52]
months = np.arange(1,13)

lgv_rows['month'] = pd.cut(lgv_rows['week_no'], bins, labels=months)

In [480]:
# drop weeks column
lgv_rows = lgv_rows.drop(columns=['week_no'])

In [481]:
lgv_rows.head()

,region,line_code,works_duration,year,comms_works,land_works,modernisation_works,other_works,overhead_works,signal_works,track_works,month
0,Rhône-Alpes,900000,25,2020,0,0,0,0,0,0,1,9
12,Franche-Comté,830000,2,2020,0,0,1,0,0,0,0,9
14,Bretagne,441000,4,2020,0,0,0,0,0,0,1,9
18,Bretagne,470000,6,2020,0,0,0,0,1,0,0,9
22,Champagne-Ardenne,70000,8,2020,0,0,0,0,0,1,0,9


We must either record cumulative **works_duration** for each region-line_code combination to ensure that the information is not devalued. However, this would not account for the dirsuption associated with each works type. Therefore, where a *1* is recorded under a works type, we will record the works duration. Consequently, each route will eventually have the cumulative disruption on each of the lines it follows for each month and each works type.

In [484]:
output_list = []

region = lgv_rows['region'].unique()
line_code = lgv_rows['line_code'].unique()
year = lgv_rows['year'].unique()
month = lgv_rows['month'].unique()

for r in region:
    print(f'{r} processing')
    for l in line_code:
        for y in year:
            for m in month:

                temp = lgv_rows[(lgv_rows['region']==r) & 
                         (lgv_rows['line_code']==l) & 
                        (lgv_rows['year']==y) & 
                        (lgv_rows['month']==m)].reset_index()

                details = temp.loc[:,['comms_works','land_works','modernisation_works','other_works','overhead_works','signal_works','track_works']]. \
                           multiply(temp.loc[:, 'works_duration'], axis="index").sum()

                output = (y,m,r,l, details[0], details[1], details[2], details[3], details[4], details[5], details[6])

                output_list.append(output)

Rhône-Alpes processing
Franche-Comté processing
Bretagne processing
Champagne-Ardenne processing
Bourgogne processing
Languedoc-Roussillon processing
Lorraine processing
Provence-Alpes-Côte-d'Azur processing
Île-de-France processing
Pays de la Loire processing
Alsace processing
Midi-Pyrénées processing
Poitou-Charentes processing
Aquitaine processing
Auvergne processing
Centre processing
Nord-Pas-de-Calais processing


In [486]:
maintenance_agg = pd.DataFrame(output_list, columns=['year','month','region','line_code','comms_works','land_works','modernisation_works','other_works','overhead_works','signal_works','track_works'])

In [487]:
maintenance_agg.head()

,year,month,region,line_code,comms_works,land_works,modernisation_works,other_works,overhead_works,signal_works,track_works
0,2020,9,Rhône-Alpes,900000,0.0,12.0,0.0,117.0,63.0,38.0,90.0
1,2020,1,Rhône-Alpes,900000,0.0,0.0,0.0,82.0,17.0,17.0,32.0
2,2020,2,Rhône-Alpes,900000,0.0,0.0,0.0,79.0,21.0,16.0,35.0
3,2020,3,Rhône-Alpes,900000,0.0,8.0,0.0,184.0,42.0,17.0,44.0
4,2020,4,Rhône-Alpes,900000,2.0,6.0,0.0,71.0,12.0,16.0,28.0


We see that works durations are now aggregated by month as intended. The maintenance dataset is now ready to be exported.

In [488]:
# export updated dataset
maintenance_agg.to_csv('data/maintenance_finalised.csv', index=False)